In [1]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 3724 (delta 14), reused 23 (delta 8), pack-reused 3684
Receiving objects: 100% (3724/3724), 8.14 MiB | 5.48 MiB/s, done.
Resolving deltas: 100% (2333/2333), done.


In [2]:
!pip install ./fastText

Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2855928 sha256=4a5d1bf6b92dff0bbf48fedb06621f891173ef29fbb75fe0216c24d25462c5e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-72uh6bsz/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [41]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore') 
ft = fasttext.load_model('cc.en.300.bin')

In [42]:
fasttext.util.reduce_model(ft, 150)

In [78]:
import numpy as np
import pandas as pd
import nltk
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import FeatureUnion
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer

In [6]:
data_url = 'https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv'
data = pd.read_csv(data_url, sep='\t')
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
class FastTextVectorizer(object):

    def __init__(self, model):
        self.model = model
        self.dim = model.get_dimension()

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([self.model.get_word_vector(text) for text in X])

In [0]:
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
def eval_pred(y_test, y_pred):
    print(f'Precision: {precision_score(y_test, y_pred, average="macro"):.2f}')
    print(f'Recall: {recall_score(y_test, y_pred, average="macro"):.2f}')
    print(f'F1-measure {f1_score(y_test, y_pred, average="macro"):.2f}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

In [70]:
clf = Pipeline([
    ("fasttext vectorizer", FastTextVectorizer(ft)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
eval_pred(y_test, y_pred)

Precision: 0.50
Recall: 0.49
F1-measure 0.47
Accuracy: 0.49


In [100]:
clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
eval_pred(y_test, y_pred)

Precision: 0.69
Recall: 0.69
F1-measure 0.69
Accuracy: 0.69


In [98]:
clf = Pipeline([
    ('feats', FeatureUnion([
        ('tfidf', TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)),
        ("fasttext vectorizer", FastTextVectorizer(ft))])),
    ('clf', SVC(class_weight="balanced", random_state=42))
    ])

clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
eval_pred(y_test, y_pred)

Precision: 0.66
Recall: 0.66
F1-measure 0.66
Accuracy: 0.66


In [99]:
clf = Pipeline([
    ('feats', FeatureUnion([
        ('tfidf', TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)),
        ("fasttext vectorizer", FastTextVectorizer(ft))])),
    ('clf',  BaggingClassifier(base_estimator=SVC(class_weight="balanced", random_state=42), n_estimators=10, max_samples=0.8, max_features=0.8))
    ])

clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
eval_pred(y_test, y_pred)

Precision: 0.66
Recall: 0.65
F1-measure 0.65
Accuracy: 0.65


Как видим, само по себе использование fasttext не даёт никаких особо интересных результатов. И вообще ExtraTreesClassifier с tf-idf работает лучше всего.

In [0]:
def adaboost_pipeline(X, y, base_model, n_est=100, fast_text=False):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

  ada = AdaBoostClassifier(base_estimator=base_model, n_estimators=n_est, random_state=1, learning_rate=0.8)

  if fast_text:
    vectorizer = FastTextVectorizer(ft)
  else:
    vectorizer = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)

  clf = Pipeline([
    ('vectorizer', vectorizer),
    ("adaboost", ada)])
  
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  eval_pred(y_test, y_pred)

In [130]:
adaboost_pipeline(X, y, ExtraTreesClassifier(n_estimators=70, max_depth=25, criterion='gini', min_samples_split=4))

Precision: 0.72
Recall: 0.71
F1-measure 0.71
Accuracy: 0.70


In [131]:
adaboost_pipeline(X, y, ExtraTreesClassifier(n_estimators=70, max_depth=25, criterion='gini', min_samples_split=4), fast_text=True)

Precision: 0.45
Recall: 0.46
F1-measure 0.45
Accuracy: 0.46


In [0]:
def xgboost_pipeline(X, y, fast_text=False):
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

  xg = xgb.XGBClassifier(objective='multi:softmax')

  if fast_text:
    vectorizer = FastTextVectorizer(ft)
  else:
    vectorizer = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)
    
  clf = Pipeline([
    ('vectorizer', vectorizer),
    ("XGB", xg)])
  
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  eval_pred(y_test, y_pred)

In [139]:
xgboost_pipeline(X, y)

Precision: 0.63
Recall: 0.63
F1-measure 0.63
Accuracy: 0.63


In [140]:
xgboost_pipeline(X, y, fast_text=True)

Precision: 0.49
Recall: 0.50
F1-measure 0.49
Accuracy: 0.49


In [0]:
def bagging_model(X, y, base_model, fast_text=False, n_est=50):
  clf = BaggingClassifier(base_estimator=base_model, n_estimators=n_est, max_samples=0.7, max_features=0.7)
    
  if fast_text:
    vectorizer = FastTextVectorizer(ft)
  else:
    vectorizer = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=150)
  
  clf = Pipeline([
    ('vectorizer', vectorizer),
    ("bagging", clf)])
  
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  eval_pred(y_test, y_pred)

In [149]:
bagging_model(X, y, ExtraTreesClassifier(n_estimators=70, max_depth=30, criterion='entropy', min_samples_split=2), n_est=20)

Precision: 0.70
Recall: 0.69
F1-measure 0.69
Accuracy: 0.69


In [150]:
bagging_model(X, y, ExtraTreesClassifier(n_estimators=70, max_depth=30, criterion='entropy', min_samples_split=2), n_est=20, fast_text=True)

Precision: 0.53
Recall: 0.51
F1-measure 0.48
Accuracy: 0.51


В результате получается, что самый высокий результат у AdaBoost. fasttext не даёт хорошего результата. Возможно, здесь стоит использовать какие-то еще фичи.